In [15]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

# Import the requests library.
import requests

# Import the datetime module from the datetime library.
from datetime import datetime

In [16]:
#2. Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
coordinates = list(lat_lngs)

In [17]:
#3. Get the nearest city using the citipy module.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

762

In [18]:
# 4. Perform an API call with the OpenWeatherMap.
from config import weather_api_key
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f0cb148ad20ab81ce62a794ee2720865


In [19]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [21]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_wdesc = city_weather["weather"][0]["description"]

        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_wdesc})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

    # Indicate that Data Loading is complete.
    print("-----------------------------")
    print("Data Retrieval Complete      ")
    print("-----------------------------")

Processing Record 1 of Set 1 | barrow
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 2 of Set 1 | rocha
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 3 of Set 1 | mrirt
City not found. Skipping...
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 4 of Set 1 | cabo san lucas
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 5 of Set 1 | vaini
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 6 of Set 1 | atuona
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 7 of Set 1 | mataura
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 8 of Set 1 | thompson
-----------------------------


In [22]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Barrow,US,71.2906,-156.7887,30.22,80,90,14.97,overcast clouds
1,Rocha,UY,-34.4833,-54.3333,53.92,80,94,6.24,moderate rain
2,Cabo San Lucas,MX,22.8909,-109.9124,86.50,88,90,4.61,overcast clouds
3,Vaini,TO,-21.2000,-175.2000,77.16,83,75,12.66,broken clouds
4,Atuona,PF,-9.8000,-139.0333,78.13,73,20,20.78,few clouds
5,Mataura,NZ,-46.1927,168.8643,53.47,79,100,3.02,overcast clouds
6,Thompson,CA,55.7435,-97.8558,62.38,48,32,3.09,scattered clouds
7,Manyana,BW,-23.4000,21.7167,53.62,24,0,9.73,clear sky
8,Sisimiut,GL,66.9395,-53.6735,34.66,77,29,9.80,scattered clouds
9,Cidreira,BR,-30.1811,-50.2056,62.91,76,93,8.43,overcast clouds


In [23]:
# Create the output file (CSV).
output_data_file = "weather_database/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")